# Importing section

In [4]:
import urllib.request
import re
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from pyspark.sql import SparkSession

In [5]:
warnings.filterwarnings("ignore")

# Getting log and initialize spark

In [6]:
urllib.request.urlretrieve("http://raw.githubusercontent.com/elastic/examples/master/Common%20Data%20Formats/apache_logs/apache_logs","apache_logs")

spark = SparkSession.builder.getOrCreate()
spark
sc = spark.sparkContext

('apache_logs', <http.client.HTTPMessage at 0x134e824d0>)